Markdown Shit

This book will detail fitting - focus first on getting a general example, sum of all prescriptions, then following books will look at specific examples

In order to manage the data downloaded from https://data.medicaid.gov/datasets?theme%5B0%5D=State+Drug+Utilization the CSV files were brought into MySQL initially using command line as MySQL was not able to handle the size of the files. 

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from pandas.tseries.offsets import QuarterEnd
# from datetime import datetime, date, time
from statsmodels.api import tsa
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.express as px
from prophet import Prophet
from prophet.plot import plot_plotly
# WARNING WARNING WARNING
import warnings
warnings.filterwarnings("ignore")
# WARNING WARNING WARNING

In [ ]:
# import data exported from Book 1
parent_df = pd.read_csv('data/quarterly_rx_state.csv',index_col=0).T

# make sure data look as expected
parent_df.head(3)

In [ ]:
parent_df.drop(['TN','WA','SD'],axis=1,inplace=True)
parent_df.drop(['09/30/2022'],axis=0,inplace=True)

In [ ]:
# find total for each date

# instantiate empty dict 
summary_dict = {}

# loop through rows
for i in parent_df.index:
    # for each date add sum of that quarter to dict
    summary_dict[i] = parent_df.loc[i].sum()

# convert to a dataframe
summary_df = pd.DataFrame.from_dict(summary_dict, orient='index', columns=['Prescriptions'])

# ensure that index is datetime type
summary_df.index = pd.to_datetime(summary_df.index)

In [ ]:
layout = go.Layout(xaxis={'title': 'Year'}, yaxis={'title': 'Prescriptions'}, title={'text': 'Prescriptions Reimbursed Annually, 1991-2022'})
fig = go.Figure(data=go.Scatter(x = list(summary_df.index), y = list(summary_df['Prescriptions']), mode='markers'),layout=layout)
fig.update()
fig.show()

Let's do some classic time series analysis, trend, seasonal, and residual decomposition



In [ ]:
# analysis_dict = {}
analysis_df_full = pd.DataFrame.empty


decomposition = tsa.seasonal_decompose(summary_df, model='additive')

analysis_dict={'trend':dict(decomposition.trend),'seasonal':dict(decomposition.seasonal),'residual':dict(decomposition.resid)}
analysis_df_full = pd.DataFrame(analysis_dict)


fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(
    name = 'Trend',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['trend']),
), row=1, col=1)
fig.append_trace(go.Scatter(
    name = 'Seasonal',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['seasonal']),
), row=2, col=1)
fig.append_trace(go.Scatter(
    name = 'Residuals',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['residual']),
), row=3, col=1)
fig.update_layout(height=600, width=800, title_text=f"Prescription Stacked Subplots")
fig.show()

Let's try differencing by quarters.

In [ ]:
# differencing 1 quarter

increment_1_df = summary_df.copy(deep=True)

new_prescription_values = []

for h, i in enumerate(increment_1_df.index):
    if h == 0:
        pass
    else:
        increment = i-QuarterEnd(1)
        new_prescription_values.append(increment_1_df.loc[i]['Prescriptions'] - increment_1_df.loc[increment]['Prescriptions'])

increment_1_df.drop(increment_1_df.index[0],axis=0,inplace=True)
increment_1_df['Prescriptions'] = new_prescription_values

layout = go.Layout(xaxis={'title': 'Year'}, yaxis={'title': 'Prescriptions'}, title={'text': 'Prescriptions Reimbursed Annually, 1991-2022'})
fig = go.Figure(data=go.Scatter(x = list(increment_1_df.index), y = list(increment_1_df['Prescriptions']), mode='markers'),layout=layout)
fig.update()
fig.show()

# analysis_dict = {}
analysis_df_full = pd.DataFrame.empty


decomposition = tsa.seasonal_decompose(increment_1_df, model='additive')

analysis_dict={'trend':dict(decomposition.trend),'seasonal':dict(decomposition.seasonal),'residual':dict(decomposition.resid)}
analysis_df_full = pd.DataFrame(analysis_dict)


fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(
    name = 'Trend',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['trend']),
), row=1, col=1)
fig.append_trace(go.Scatter(
    name = 'Seasonal',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['seasonal']),
), row=2, col=1)
fig.append_trace(go.Scatter(
    name = 'Residuals',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['residual']),
), row=3, col=1)
fig.update_layout(height=600, width=800, title_text=f"Prescription Stacked Subplots")
fig.show()

In [ ]:
# differencing 2 quarters

increment_2_df = summary_df.copy(deep=True)

new_prescription_values = []

for h, i in enumerate(increment_2_df.index):
    if h < 2:
        pass
    else:
        increment = i-QuarterEnd(2)
        new_prescription_values.append(increment_2_df.loc[i]['Prescriptions'] - increment_2_df.loc[increment]['Prescriptions'])

increment_2_df.drop(increment_2_df.index[0:2],axis=0,inplace=True)
increment_2_df['Prescriptions'] = new_prescription_values

layout = go.Layout(xaxis={'title': 'Year'}, yaxis={'title': 'Prescriptions'}, title={'text': 'Prescriptions Reimbursed Annually, 1991-2022'})
fig = go.Figure(data=go.Scatter(x = list(increment_2_df.index), y = list(increment_2_df['Prescriptions']), mode='markers'),layout=layout)
fig.update()
fig.show()

# analysis_dict = {}
analysis_df_full = pd.DataFrame.empty


decomposition = tsa.seasonal_decompose(increment_2_df, model='additive')

analysis_dict={'trend':dict(decomposition.trend),'seasonal':dict(decomposition.seasonal),'residual':dict(decomposition.resid)}
analysis_df_full = pd.DataFrame(analysis_dict)


fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(
    name = 'Trend',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['trend']),
), row=1, col=1)
fig.append_trace(go.Scatter(
    name = 'Seasonal',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['seasonal']),
), row=2, col=1)
fig.append_trace(go.Scatter(
    name = 'Residuals',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['residual']),
), row=3, col=1)
fig.update_layout(height=600, width=800, title_text=f"Prescription Stacked Subplots")
fig.show()

In [ ]:
# differencing 3 quarters

increment_3_df = summary_df.copy(deep=True)

new_prescription_values = []

for h, i in enumerate(increment_3_df.index):
    if h < 3:
        pass
    else:
        increment = i-QuarterEnd(2)
        new_prescription_values.append(increment_3_df.loc[i]['Prescriptions'] - increment_3_df.loc[increment]['Prescriptions'])

increment_3_df.drop(increment_3_df.index[0:3],axis=0,inplace=True)
increment_3_df['Prescriptions'] = new_prescription_values

layout = go.Layout(xaxis={'title': 'Year'}, yaxis={'title': 'Prescriptions'}, title={'text': 'Prescriptions Reimbursed Annually, 1991-2022'})
fig = go.Figure(data=go.Scatter(x = list(increment_3_df.index), y = list(increment_3_df['Prescriptions']), mode='markers'),layout=layout)
fig.update()
fig.show()

# analysis_dict = {}
analysis_df_full = pd.DataFrame.empty


decomposition = tsa.seasonal_decompose(increment_3_df, model='additive')

analysis_dict={'trend':dict(decomposition.trend),'seasonal':dict(decomposition.seasonal),'residual':dict(decomposition.resid)}
analysis_df_full = pd.DataFrame(analysis_dict)


fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(
    name = 'Trend',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['trend']),
), row=1, col=1)
fig.append_trace(go.Scatter(
    name = 'Seasonal',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['seasonal']),
), row=2, col=1)
fig.append_trace(go.Scatter(
    name = 'Residuals',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['residual']),
), row=3, col=1)
fig.update_layout(height=600, width=800, title_text=f"Prescription Stacked Subplots")
fig.show()

In [ ]:
# differencing 4 quarters

increment_4_df = summary_df.copy(deep=True)

new_prescription_values = []

for h, i in enumerate(increment_4_df.index):
    if h < 4:
        pass
    else:
        increment = i-QuarterEnd(2)
        new_prescription_values.append(increment_4_df.loc[i]['Prescriptions'] - increment_4_df.loc[increment]['Prescriptions'])

increment_4_df.drop(increment_4_df.index[0:4],axis=0,inplace=True)
increment_4_df['Prescriptions'] = new_prescription_values

layout = go.Layout(xaxis={'title': 'Year'}, yaxis={'title': 'Prescriptions'}, title={'text': 'Prescriptions Reimbursed Annually, 1991-2022'})
fig = go.Figure(data=go.Scatter(x = list(increment_4_df.index), y = list(increment_4_df['Prescriptions']), mode='markers'),layout=layout)
fig.update()
fig.show()

# analysis_dict = {}
analysis_df_full = pd.DataFrame.empty


decomposition = tsa.seasonal_decompose(increment_4_df, model='additive')

analysis_dict={'trend':dict(decomposition.trend),'seasonal':dict(decomposition.seasonal),'residual':dict(decomposition.resid)}
analysis_df_full = pd.DataFrame(analysis_dict)


fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(
    name = 'Trend',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['trend']),
), row=1, col=1)
fig.append_trace(go.Scatter(
    name = 'Seasonal',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['seasonal']),
), row=2, col=1)
fig.append_trace(go.Scatter(
    name = 'Residuals',
    x = list(analysis_df_full.index),
    y = list(analysis_df_full['residual']),
), row=3, col=1)
fig.update_layout(height=600, width=800, title_text=f"Prescription Stacked Subplots")
fig.show()

In [ ]:
# autocorrelation function

plt.figure(figsize=(15, 5))
plot_acf(summary_df, lags=15, ax=plt.gca())
plt.xlabel('Lag')
plt.ylabel('AC')
plt.show()

In [ ]:
# partial autocorrelation function

plt.figure(figsize=(15, 5))
plot_pacf(summary_df, lags=15, ax=plt.gca(), method='ywm')
plt.xlabel('Lag')
plt.ylabel('Partial AC')
plt.show()

It looks like differencing will be important. Let's get a baseline - just the mean - and calculate MAPE and SMAPE

In [ ]:
def mean_absolute_percentage_error(true_values, predicted_values):
    """
    Calculate the mean absolute percentage error. 
    Find the prediction error and divide by the true value, then average.
    """
    
    error = true_values - predicted_values
    absolute_percentage_error = np.abs(error/true_values)
    mape = round((absolute_percentage_error.mean() * 100),2)
    
    return mape

def symmetric_mean_absolute_percentage_error(true_values, predicted_values):
    numerator = abs(predicted_values-true_values)
    denominator = abs(true_values)+abs(predicted_values)
    division = numerator/denominator
    summed_div = division.sum()
    smape = round(((100/(len(true_values)))*summed_div),2)
    return smape

In [ ]:
print(f"The mean quarterly number of prescriptions for all available data is {round(summary_df['Prescriptions'].mean(),0)}, plotted as a red line below.")

layout = go.Layout(xaxis={'title': 'Year'}, yaxis={'title': 'Prescriptions'}, title={'text': 'Prescriptions Reimbursed Quarterly, 1991-2022'})
fig = go.Figure(data=go.Scatter(x = list(summary_df.index), y = list(summary_df['Prescriptions']), mode='markers'),layout=layout)
fig.add_hline(y=summary_df['Prescriptions'].mean(), line_color='red')
fig.show()

mean_prescriptions = pd.DataFrame.from_dict({i: 136620699 for i in summary_df.index}, orient='index', columns=['Prescriptions'])

print(f"Using mean prescriptions, the MAPE is {mean_absolute_percentage_error(summary_df['Prescriptions'],mean_prescriptions['Prescriptions'])}%;\
 the SMAPE is {symmetric_mean_absolute_percentage_error(summary_df['Prescriptions'],mean_prescriptions['Prescriptions'])}%.")

In [ ]:
# finding parameters

cutoff = '2020-12-31'
cutoff = pd.to_datetime(cutoff)

train_dict = {}
test_dict = {}
run_num = 0

for p in range(0,11):
    for d in range(0,11):
        for q in range(0,11):
            print(f"Running series p {p}, d {d}, q {q}",end="\r")
            try:
                predict_df = summary_df.copy(deep=True)
                # predict_df.set_index('date',inplace=True)
                predict_df.index = pd.to_datetime(predict_df.index)
                first_day = predict_df.index.min()
                last_day = predict_df.index.max()
                full_range = pd.date_range(start=first_day, end=last_day, freq="Q")
                # predict_df = predict_df.reindex(full_range)
                predict_df.fillna(method='ffill',inplace=True)

                train = predict_df.loc[predict_df.index <= cutoff].dropna()
                test = predict_df.loc[predict_df.index > cutoff]

                model = SARIMAX(train, order=(p, d, q), trend="c")
                model_fit = model.fit(disp=0)

                # model_fit.summary()

                predictions = model_fit.predict(start=0, end=len(train)+len(test)-1)
                
                train_smape = symmetric_mean_absolute_percentage_error(pd.Series(list(train['Prescriptions'])), pd.Series(list(predictions[train.index])))
                test_smape = symmetric_mean_absolute_percentage_error(pd.Series(list(test['Prescriptions'])), pd.Series(list(predictions[test.index])))

                train_dict[run_num] = {'p':p,'d':d,'q':q,'val':train_smape}
                test_dict[run_num] = {'p':p,'d':d,'q':q,'val':test_smape}
                run_num+=1
            except:
                pass


train_vals_df = (pd.DataFrame(train_dict).T)
test_vals_df = (pd.DataFrame(test_dict).T)
px.scatter_3d(train_vals_df, x='p',y='d',z='q',color='val',range_color=[6,9]).show()

In [ ]:
# RUN select hyperparameters
combined_dict = {}
for i in train_vals_df.index: # make a list of places where they overlap then plot
    if 0 < train_vals_df.loc[i,'val'] < 100 and 0 < test_vals_df.loc[i,'val'] < 100:
        combined_dict[i] = {'p':train_vals_df.loc[i,'p'],
                            'd':train_vals_df.loc[i,'d'],
                            'q':train_vals_df.loc[i,'q'],
                            'val':(train_vals_df.loc[i,'val']+test_vals_df.loc[i,'val'])/2}
combined_df = pd.DataFrame(combined_dict).T
for i in combined_df.index:
    if combined_df.loc[i,'val'] > combined_df['val'].mean():
        combined_df.drop(index=i,inplace=True)
high_threshold = combined_df.describe()['val'][4]
low_threshold = combined_df.describe()['val'][3]

px.scatter_3d(combined_df, x='p',y='d',z='q',color='val',range_color=[low_threshold,high_threshold]).show()

display(combined_df.sort_values('val'))

In [ ]:
p, d, q = 10, 1, 6

predict_df = summary_df.copy(deep=True)
predict_df.index = pd.to_datetime(predict_df.index)
first_day = predict_df.index.min()
last_day = predict_df.index.max()
full_range = pd.date_range(start=first_day, end=last_day, freq="Q")
predict_df.fillna(method='ffill',inplace=True)

train = predict_df.loc[predict_df.index <= cutoff].dropna()
test = predict_df.loc[predict_df.index > cutoff]

model = SARIMAX(train, order=(p, d, q), trend="c")
model_fit = model.fit(disp=0)


predictions = model_fit.predict(start=0, end=len(train)+len(test)-1)

train_smape = mean_absolute_percentage_error(pd.Series(list(train['Prescriptions'])), pd.Series(list(predictions[train.index])))
test_smape = mean_absolute_percentage_error(pd.Series(list(test['Prescriptions'])), pd.Series(list(predictions[test.index])))

display(model_fit.summary())

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(train.index), y = train['Prescriptions'], name = 'Train'))
fig.add_trace(go.Scatter(x=list(test.index), y = test['Prescriptions'], name = 'Test'))
fig.add_trace(go.Scatter(x=list(predictions.index), y = predictions, name = 'Fit'))
fig.update_layout(title=f"Prescriptions Annually<Br>p = {p}, d = {d}, q = {q},")
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(xaxis_title = 'Year', yaxis_title='Prescriptions')
fig.show()

print(f"The train SMAPE is {train_smape}% and the test SMAPE is {test_smape}% with the parameters p = {p}, d = {d}, q = {q}.")

In [464]:
# display avg of top parameters
top_combined = combined_df.sort_values('val').index[0:6]
avgd_fit_dict = {}
for ind in top_combined:
    p, d, q = combined_df.loc[ind,'p'], combined_df.loc[ind,'d'], combined_df.loc[ind,'q']
    predict_df = summary_df.copy(deep=True)
    predict_df.index = pd.to_datetime(predict_df.index)
    first_day = predict_df.index.min()
    last_day = predict_df.index.max()
    full_range = pd.date_range(start=first_day, end=last_day, freq="Q")
    predict_df = predict_df.reindex(full_range)
    predict_df.fillna(method='ffill',inplace=True)

    train = predict_df.loc[predict_df.index <= cutoff].dropna()
    test = predict_df.loc[predict_df.index > cutoff]

    model = SARIMAX(train, order=(p, d, q), trend="c")
    model_fit = model.fit(disp=0)

    predictions = model_fit.predict(start=0, end=len(train)+len(test)-1)

    avgd_fit_dict[f'p{p}, d{d}, q{q}'] = predictions

avgd_fit_df = pd.DataFrame(avgd_fit_dict, index = list(predictions.index))
avgd_fit_df['mean'] = avgd_fit_df.mean(axis=1)


train_smape = mean_absolute_percentage_error(pd.Series(list(avgd_fit_df['mean'][0:len(predictions[train.index])])), pd.Series(list(predictions[train.index])))
test_smape = mean_absolute_percentage_error(pd.Series(list(avgd_fit_df['mean'][len(predictions[train.index]):])), pd.Series(list(predictions[test.index])))

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(train.index), y = train['Prescriptions'], name = 'Train'))
fig.add_trace(go.Scatter(x=list(test.index), y = test['Prescriptions'], name = 'Test'))
fig.add_trace(go.Scatter(x=list(avgd_fit_df.index), y = avgd_fit_df['mean'], name = 'Averaged Fit'))
fig.update_layout(title="Prescriptions Annually<Br>Averaged Fit")
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(xaxis_title = 'Year', yaxis_title='Prescriptions')
fig.show()

print(f"The train SMAPE is {train_smape}% and the test SMAPE is {test_smape}% with the top 5 predictions averaged.")

The train SMAPE is 2.23% and the test SMAPE is 3270.44% with the top 5 predictions averaged.


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

p_param = 2

model = SARIMAX(train, order=(p_param, 0, 0), trend="c")
model_fit = model.fit(disp=0)

model_fit.summary()

# Facebook Prophet

We can also take a look at the Facebook Prophet tool.

In [465]:
start = '03/31/1991'
end = '12/31/2020'

start = pd.to_datetime(start)
end = pd.to_datetime(end)

for x in [1,1000]:
    predict_df = summary_df.copy(deep=True)
    predict_df.reset_index(inplace=True,names=['date'])
    predict_df['date'] = pd.to_datetime(predict_df['date'])
    prophet_train = predict_df.loc[predict_df['date'] <= end]
    prophet_test = predict_df.loc[predict_df['date'] > end]
    prophet_train = prophet_train.loc[prophet_train['date'] >= start]
    prophet_train.rename(columns={"date": "ds", 'Prescriptions': "y"}, inplace=True)
    prophet_test.rename(columns={"date": "ds", 'Prescriptions': "y"}, inplace=True)
    model = Prophet(yearly_seasonality=4,changepoint_prior_scale=(.5*x))
    model.fit(prophet_train)
    future = model.make_future_dataframe(periods=(len(predict_df)-len(predict_df.loc[predict_df['date'] <= end])), freq="Q", include_history=True)
    forecast = model.predict(future)
    fig = plot_plotly(model, forecast)
    fig.show()
    predict_train = forecast['yhat'][0:len(model.history)]
    predict_test = forecast['yhat'][len(model.history):]
    real_train = predict_df['Prescriptions'][0:len(model.history)]
    real_test = predict_df['Prescriptions'][len(model.history):]

    train_smape = symmetric_mean_absolute_percentage_error(real_train, predict_train)
    test_smape = symmetric_mean_absolute_percentage_error(real_test, predict_test)
    print(f"The train SMAPE is {train_smape}% and the test SMAPE is {test_smape}%")

10:42:31 - cmdstanpy - INFO - Chain [1] start processing
10:42:31 - cmdstanpy - INFO - Chain [1] done processing


10:42:31 - cmdstanpy - INFO - Chain [1] start processing
10:42:32 - cmdstanpy - INFO - Chain [1] done processing


The train SMAPE is 3.06% and the test SMAPE is 1.89%


The train SMAPE is 1.89% and the test SMAPE is 2.44%


# CNN